# COCO inference+eval

In [1]:
import os
import numpy as np
import torch, gc
from PIL import Image

from pycocotools.coco import COCO
# import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

import torchvision
from torchvision.io import read_image
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights, SSD300_VGG16_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import json

from torchvision.utils import draw_bounding_boxes
from torchvision.ops import masks_to_boxes

import torchvision.transforms.functional as F

from pycocotools.cocoeval import COCOeval

from datetime import datetime

In [2]:
# eval_name = 'ori_eval.json'
# eval_name = '0.05_eval.json' 
# eval_name = '0.10_eval.json' 
# eval_name = '0.15_eval.json' 
# eval_name = '0.20_eval.json' 
eval_name = '0.25_eval.json'
# eval_name = '0.30_eval.json'

# model = torchvision.models.detection.ssd300_vgg16(weights='DEFAULT')
# model = torch.load("/home/Joe/MY/CV/CV_FINAL/now/model/0.05.pth")
# model = torch.load("/home/Joe/MY/CV/CV_FINAL/now/model/0.10.pth")
# model = torch.load("/home/Joe/MY/CV/CV_FINAL/now/model/0.15.pth")
# model = torch.load("/home/Joe/MY/CV/CV_FINAL/now/model/0.20.pth")
model = torch.load("/home/Joe/MY/CV/CV_FINAL/now/model/0.25.pth")
# model = torch.load("/home/Joe/MY/CV/CV_FINAL/now/model/0.30.pth")

device = torch.device('cuda')
model.to(device)

SSD(
  (backbone): SSDFeatureExtractorVGG(
    (features): Sequential(
      (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True

In [3]:
# get rid of zero annotation samples
# pick 10000 imgs from differnet type
# => 125 imgs per type
def no_zero_imgs(train):

    no_zero_img = []

    if train == True:
        coco = COCO('../coco/annotations/instances_train2017.json')
        img_load = coco.loadImgs(coco.getImgIds())
        cat_type = coco.getCatIds()
        for i in cat_type:
            img_id_list = coco.getImgIds(catIds = i)
            for j in range(125):
                img_id = img_id_list[j]

                if( len(coco.getAnnIds(img_id)) == 0 ):
                    continue
                no_zero_img.append(coco.loadImgs(ids= img_id)[0])

    else:
        coco = COCO('/home/Joe/MY/CV/CV_FINAL/anchor_pruning/data/coco/annotations/instances_val2017.json')
        img_load = coco.loadImgs(coco.getImgIds())
        img_id_list = coco.getImgIds()
        for i in range(len(img_load)):
            img_id = img_id_list[i]
            if( len(coco.getAnnIds(img_id)) == 0 ):
                continue
            no_zero_img.append(coco.loadImgs(ids= img_id)[0])


    return no_zero_img


class CocoDataset(torch.utils.data.Dataset):
    def __init__(self, train):

        if train == True:
            self.ann_file = "../coco/annotations/instances_train2017.json"
            self.imgs_dir = '../coco/images/train2017/'
        else:
            self.ann_file = '/home/Joe/MY/CV/CV_FINAL/anchor_pruning/data/coco/annotations/instances_val2017.json'
            self.imgs_dir = '/home/Joe/MY/CV/CV_FINAL/anchor_pruning/data/coco/val2017/'

        self.coco = COCO(self.ann_file)
        self.img_load = no_zero_imgs(train)

    def __getitem__(self, idx):

        # process for images

        img_name = self.img_load[idx]['file_name']
        dir = self.imgs_dir + "/" + img_name
        preprocess = SSD300_VGG16_Weights.COCO_V1.transforms()
        Image = preprocess(read_image(dir))
        img = Image
        # for targets

        img_id = self.img_load[idx]['id']
        ann_ids = self.coco.getAnnIds(img_id)
        num_detect = len(ann_ids)

        d = {}
        if  num_detect == 0:
            print("wrong!", img_id)


        for j in range (num_detect):

            box_now = self.coco.loadAnns(ann_ids)[j]['bbox']

            new_box = torch.tensor( [box_now[0], box_now[1], box_now[0] + box_now[2], box_now[1] + box_now[3]] ).reshape(1, 4)
            new_label = torch.tensor( self.coco.loadAnns(ann_ids)[j]['category_id'] ).reshape(1)

            if j == 0:
                temp_box = new_box
                temp_label = new_label
            else:
                temp_box = torch.cat((temp_box, new_box), dim = 0)
                temp_label = torch.cat((temp_label, new_label))

        target = {}
        target['boxes'] = temp_box
        target['labels'] = temp_label
        target['image_id'] = img_id
        return img, target

    def __len__(self):
        return len(self.img_load)

def custom_collate(data):
    return data

# COCO inference

In [4]:
test = torch.utils.data.DataLoader(
    CocoDataset(train=False),
    batch_size = 1,
    shuffle = False,
    collate_fn = custom_collate,
    pin_memory = False )

len(test)

loading annotations into memory...
Done (t=0.28s)
creating index...
index created!
loading annotations into memory...
Done (t=0.25s)
creating index...
index created!


4952

In [5]:
import time

start = time.time()
bbox_result = []
for i, data in enumerate(test):

    model.eval()
    output = model([data[0][0].cuda()])

    for j in range(len(output[0]['boxes'])):
        temp_dic = {}
        temp_dic['image_id'] = data[0][1]['image_id']
        temp_dic['category_id'] = output[0]['labels'][j].item()
        out_list = output[0]['boxes'][j].tolist()

        out_list[2] = out_list[2] - out_list[0]
        out_list[3] = out_list[3] - out_list[1]

        out_list = [round(num, 2) for num in out_list]
        temp_dic['bbox'] = out_list

        temp_dic['score'] = round(output[0]['scores'][j].item(), 3)

        bbox_result.append(temp_dic)

    # if i % 50 == 0:
    #     print(i)
end = time.time()

exe_time = end-start

with open(eval_name, "w") as outfile:
    json.dump(bbox_result, outfile)

print("done")

done


## COCO evaluate

In [6]:
test_ann_file = "/home/Joe/MY/CV/CV_FINAL/anchor_pruning/data/coco/annotations/instances_val2017.json"
coco_gt = COCO(test_ann_file)
prediction = coco_gt.loadRes(eval_name)

loading annotations into memory...
Done (t=0.37s)
creating index...
index created!
Loading and preparing results...
DONE (t=2.46s)
creating index...
index created!


In [7]:
cocoEval = COCOeval(coco_gt, prediction, 'bbox')
cocoEval.evaluate()
cocoEval.accumulate()

print(eval_name)
print("Execute time: " + str(exe_time))

cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=14.66s).
Accumulating evaluation results...
DONE (t=4.44s).
0.25_eval.json
Execute time: 79.42721581459045
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.033
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.062
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.030
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.011
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.034
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.060
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.059
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.079
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.083
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.026
 Average Recall     (